# Region Growing

Starting from seed points, the region-growing algorithm is used to decide whether a point should be included in a region or not. Two diffenerent implemenations are explained in this notebook:
- Region growing based on the Point Cloud Library (PCL) implementation
- Clustering based region growing using "Label Connected Components"


In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pylas
import time

from src.utils.las_utils import label_and_save_las
from src.utils.labels import Labels

In [ ]:
# In this example we want to grow the building points, the initial seed points.
# The ground points must be removed to fast and properly cluster the cloud
seed_point_label = Labels.BUILDING
exclude_label = Labels.GROUND

# Select the file to process.
in_file = "../datasets/pointcloud/labelled_2386_9702.laz"
out_file = "../datasets/pointcloud/grown_2386_9702.laz"

# Read LiDAR point cloud
las = pylas.read(in_file)
# Configure the points that we want to perform region growing on 
mask = (las.label != exclude_label)

## Region growing segmentation

### Algorithm explained
Based on: https://pcl.readthedocs.io/projects/tutorials/en/latest/region_growing_segmentation.html

With the PCL implementation, a point is picked with minimum curvature value and the region growth process is started. In our implementation, we start from initial seed points that are considered to be inside the object to be segmented. 

For every point in the seed list, the algorithm finds neighboring points. Every neighbor is tested for the angle between its normal and normal of the current seed point. If the angle is less than the threshold value then current point is added to the current region.

In [ ]:
from src.cluster.region_growing import RegionGrowing

In [ ]:
start = time.time()

# Initialize region growing class
reg = RegionGrowing()
# Process the file
reg.set_input_cloud(las, seed_point_label, mask)
regions = reg.region_growing(seed_point_label, las.label)

end = time.time()
print(f'Finished in {end-start:.2f} seconds.')

## Clustering based region growing segmentation

### Algorithm explained
This code performs clustering based region growing segmentation. The main tasks include: 
- The [Label Connected Components Segmentation](https://www.cloudcompare.org/doc/wiki/index.php?title=Label_Connected_Components) algorithm clusters the selected cloud(s) in smaller parts separated by a minimum distance.
- Next, our code grows regions, or in this case all points inside a clusters, based on a percentage of initial seed points inside a cluster. 

### Installation instructions
A Python wrapper provided for CloudCompare allows to use “Label Connected Components”. The Python wrapper works on **linux only**. This wrapper and all the dependencies can be installed using the build and installation instructions in the README at [CloudCompare-PythonPlugin.](https://github.com/tmontaigu/CloudCompare-PythonPlugin.git)

In [ ]:
from src.cluster.label_connected_comp import LabelConnectedComp

In [ ]:
start = time.time()

# Initialize clustering based region growing class
lcc = LabelConnectedComp(octree_level=10, min_component_size=100)
# Process the file
lcc.set_input_cloud(las, mask)
lcc.label_connected_comp()
regions = lcc.fill_components(las.label, seed_point_label, mask, threshold=0.1)

end = time.time()
print(f'Finished in {end-start:.2f} seconds.')

In [ ]:
# Save labelled point cloud
label_and_save_las(las, regions, out_file)